[View in Colaboratory](https://colab.research.google.com/github/jainhitesh9998/age_gender_race_detection_keras/blob/master/training_notebook_colab.ipynb)

In [1]:
!git clone https://github.com/jainhitesh9998/age_gender_race_detection_keras.git

Cloning into 'age_gender_race_detection_keras'...
remote: Counting objects: 23363, done.
remote: Compressing objects: 100% (23353/23353), done.
remote: Total 23363 (delta 9), reused 23360 (delta 6), pack-reused 0
Receiving objects: 100% (23363/23363), 154.03 MiB | 29.27 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Checking out files: 100% (33511/33511), done.


In [2]:
cd age_gender_race_detection_keras/weights/

/content/age_gender_race_detection_keras/weights


In [3]:
!wget https://github.com/Tony607/Keras_age_gender/releases/download/V1.0/weights.18-4.06.hdf5


Redirecting output to ‘wget-log’.


In [4]:
cd ..

/content/age_gender_race_detection_keras


In [5]:
!python trainer.py

Using TensorFlow backend.


In [0]:
from wide_resnet import wide_resnet
from utils import utils
import sklearn
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import os
import numpy as np
from keras.callbacks import ModelCheckpoint

def generator(samples, batch_size = 32):
    num_samples = len(samples)
    print(num_samples)
    while 1: #to run the generator indefinitely, pumping the X, Y sets for the neural network
        #shuffle the samples on each EPOCH
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset: offset + batch_size]
            labels = []
            images = []
            ages = []
            genders = []
            races = []
            age = []
            gender = []
            race = []
            for batch_sample in batch_samples:
                path, a, g, r = batch_sample
                name =  os.path.split(batch_sample[0])[-1]
                image = utils.read_image(path)
                #print(image.shape)
                if(a>100):
                  continue
                images.append(image)
                age.append(a)
                gender.append(abs(1 - g))
                race.append(r)
            ages = to_categorical(age, num_classes=101)
            genders = to_categorical(gender, num_classes=2)
            races = to_categorical(race, num_classes=5)
            labels = [np.array(genders), np.array(ages), np.array(races)]
            X_train = np.array(images)
            yield X_train,labels

def main():
    samples = utils.read_csv('extras/data.csv')
    model = wide_resnet.WideResNet(image_size=64, race=True, train_branch=True)()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mae', 'acc'])
    sklearn.utils.shuffle(samples)
    train_samples, validation_samples = train_test_split(samples, test_size=0.2)
    # Hyperparameters
    train_generator = generator(train_samples, 1)
    validation_generator = generator(validation_samples, 1)
    epoch = 10
    # end of hyperparameters
    filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    history = model.fit_generator(train_generator, samples_per_epoch=len(train_samples),
                                  validation_data=validation_generator, \
                                  nb_val_samples=len(validation_samples), nb_epoch=epoch, callbacks=callbacks_list)
    model.save(os.path.join('weights','model_new.h5'))


In [0]:
main()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., steps_per_epoch=26767, epochs=10, validation_steps=6692)`


Epoch 1/1026767

 1469/26767 [>.............................] - ETA: 29:22 - loss: 19.6178 - gender_detection_loss: 0.6683 - age_detection_loss: 4.8529 - race_loss: 13.5404 - gender_detection_mean_absolute_error: 0.4571 - gender_detection_acc: 0.5773 - age_detection_mean_absolute_error: 0.0194 - age_detection_acc: 0.0177 - race_mean_absolute_error: 0.3362 - race_acc: 0.1593